# webscrape AEMO schemas

This playbook downloads [these web pages](https://nemweb.com.au/Reports/Current/MMSDataModelReport/Electricity/MMS%20Data%20Model%20Report_files/MMS_504_9.htm) to detect the list of columns, column types, and index columns for each table in AEMO's dataset.

Each table is documented on a page between these URLs:

* https://nemweb.com.au/Reports/Current/MMSDataModelReport/Electricity/MMS%20Data%20Model%20Report_files/MMS_001.htm#1
* https://nemweb.com.au/Reports/Current/MMSDataModelReport/Electricity/MMS%20Data%20Model%20Report_files/MMS_503.htm#1

So we do a for loop to grab each one. (Ignoring pages which are other kinds of documentation.)

We want the list of "Primary Key Columns", "Index Columns" (there may be multiple sets of these). And we want "Content", with columns "Name" and "Data Type"


In [ ]:
import re
import json
import os

from tqdm import tqdm # progress bar animation
import requests # HTTP downloads
from bs4 import BeautifulSoup # webscraping

In [ ]:
base_data_dir = '/media/matthew/nemweb/AppliedEconometrics/data'

output_path = os.path.join(base_data_dir, '01-aemo-schemas.json')

In [ ]:
# create a session object to re-use between requests
# to hopefully speed up downloads by not re-doing the TLS handshake for each HTTP request
# (unsure if this actually speeds things up)
session = requests.Session()

In [ ]:
schemas = {}
for i in tqdm(range(1, 503+1)):
    url = f"https://nemweb.com.au/Reports/Current/MMSDataModelReport/Electricity/MMS%20Data%20Model%20Report_files/MMS_{i}.htm#1"
    r = session.get(url)
    r.raise_for_status()
    html = r.text
    
    # it's called "soup" because the python webscraping library is called "beautiful soup"
    soup = BeautifulSoup(html)

    h2 = soup.find('h2')
    if h2 is None:
        # not a table schema page, this is another page
        # ignore it
        # e.g. https://nemweb.com.au/Reports/Current/MMSDataModelReport/Electricity/MMS%20Data%20Model%20Report_files/MMS_2.htm#1
        continue
    h2_text = h2.text.strip()
    if not h2_text.startswith("Table: "):
        if not h2_text.strip().startswith('Package') and not h2_text.strip().startswith('Diagram'):
            print(f"Ignoring page with h2: {h2_text}: {url}")
    else:
        # watch out, sometimes the documentation has an incorrect space inside the table name
        # e.g. SET_ APC_COMPENSATION instead of SET_APC_COMPENSATION
        table_name = re.match(r"Table: (.*)", h2_text).group(1).replace(' ','')
        
        schemas[table_name] = {
            'indexes': [],
            'columns': {}
        }
        for (i, h3) in enumerate(soup.find_all('h3')):
            h3_name = h3.find('a').get('name').strip()
            h3_text = h3.text.strip()
            if i == 0:
                assert table_name == h3.text.strip().replace(' ',''), f"Table name from top h2 was {table_name}, from first h3 was {h3.text.strip()} in {url}"
                table = h3.find_next('table')
                if table is None:
                    print(f"No first table on {url}")
                schemas[table_name]['description'] = table.find_all('p')[-1].text.strip()
                
            elif h3_text == "Primary Key Columns":
                # the next element is a table.
                # grab each cell of table, except the first one
                # that's the primary key name
                schemas[table_name]['primary_keys'] = [td.text.strip() for td in h3.find_next('table').find_all('td')[1:]]
            elif h3_text == "Index Columns":
                # same again, but for an index
                schemas[table_name]['indexes'].append([td.text.strip() for td in h3.find_next('table').find_all('td')[1:]])
            elif h3_text == "Content":
                # the list of all column names
                for row in h3.find_next('table').find_all('tr')[1:]:
                    cells = row.find_all('td')
                    column_name = cells[0].text.strip()
                    column_type = cells[1].text.strip()
                    column_desc = cells[-1].text.strip()
                    schemas[table_name]['columns'][column_name] = {
                        'AEMO_type': column_type,
                        'comment': column_desc
                    }
            elif h3_text == "Description":
                # ignore
                pass
            else:
                print(f"Unknown h3: {h3.text.strip()} on {url}")
        
        # sometimes there's so many columns that there is a second page, just of the "Content" table
        # example: https://nemweb.com.au/Reports/Current/MMSDataModelReport/Electricity/MMS%20Data%20Model%20Report_files/MMS_128_1.htm
        for a in soup.find_all('a'):
            if a.get('href') and re.match(r"MMS_(\d+)_(\d+).htm", a['href']):
                try:
                    int(a.text.strip())
                except ValueError:
                    # this is not a subsequent page, it's another link
                    # e.g. to the list of tables
                    # https://nemweb.com.au/Reports/Current/MMSDataModelReport/Electricity/MMS%20Data%20Model%20Report_files/MMS_358_1.htm
                    continue
                else:
                    url_prefix, slash, url_end = url.rpartition('/')
                    new_url = url_prefix + '/' + a['href']
                    r = session.get(new_url)
                    r.raise_for_status()
                    html = r.text
                    soup = BeautifulSoup(html)
                    h3 = soup.find('h3')
                    assert h3.text.strip() == 'Content', f"Unexpected first h3 in second page: {new_url} after {url}"
                    table = h3.find_next('table')
                    for row in table.find_all('tr')[1:]:
                        cells = row.find_all('td')
                        try:
                            column_name = cells[0].text.strip()
                            column_type = cells[1].text.strip()
                            column_desc = cells[-1].text.strip()
                        except IndexError:
                            print(f"Error with {url}, {row}")
                            raise
                        schemas[table_name]['columns'][column_name] = {
                            'AEMO_type': column_type,
                            'comment': column_desc
                        }


In [ ]:
len(schemas)

In [ ]:
# manually add a schema for CO2EII_AVAILABLE_GENERATORS
# This table doesn't appear in the nemweb schema documentation
# and we might use it
# Guess the types from columns in other tables with the same name
co2eii_cols = [
    'STATIONNAME',
    'DUID',
    'GENSETID',
    'REGIONID',
    'CO2E_EMISSIONS_FACTOR',
    'CO2E_ENERGY_SOURCE',
    'CO2E_DATA_SOURCE',
]

schemas['CO2EII_AVAILABLE_GENERATORS'] = {
    'columns': {},
    'primary_keys': ['DUID'],
    'indexes': [] # none
}
for c in co2eii_cols:
    for (other_table, other_schema) in schemas.items():
        if c in other_schema['columns']:
            schemas['CO2EII_AVAILABLE_GENERATORS']['columns'][c] = other_schema['columns'][c]
            
    if c not in schemas['CO2EII_AVAILABLE_GENERATORS']['columns']:
        raise ValueError(f"Missing {c}")

In [ ]:
# there are mistakes in the schema

# change int to float
# the actual digits don't matter. Later on we treat all `NUMBER(x,y)` the same
schemas['DUDETAIL']['columns']['REGISTEREDCAPACITY']['AEMO_type'] = 'NUMBER(3,2)'
schemas['DUDETAIL']['columns']['MAXCAPACITY']['AEMO_type'] = 'NUMBER(3,2)'

For the largest tables, which are each larger than the smallest 200 tables combined, let's discard the columns we know we don't need.

In [ ]:
large_tables = ['DISPATCHLOAD', 'DISPATCHREGIONSUM']
for (table, schema) in schemas.items():
    if table in large_tables:
        to_drop = []
        cols = list(schema['columns'].keys())
        for c in cols:
            if any(c.upper().startswith(prefix) for prefix in ['RAISE', 'LOWER', 'VIOLATION', 'MARGINAL']) \
               and c not in schema['primary_keys']:
                to_drop.append(c)
        if to_drop:
            schema['columns_to_drop'] = to_drop
            print(f"Deleting columns {to_drop} from {table}")

In [ ]:
with open(output_path, 'w') as f:
    json.dump(schemas, f, indent=2)